# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232804860191                   -0.50    7.0
  2   -7.248973396387       -1.79       -1.37    1.0
  3   -7.250591001552       -2.79       -1.65    2.0
  4   -7.251005722853       -3.38       -1.87    1.0
  5   -7.251273912204       -3.57       -2.25    1.0
  6   -7.251333965205       -4.22       -2.72    1.0
  7   -7.251338128933       -5.38       -3.26    1.0
  8   -7.251338739811       -6.21       -3.61    2.0
  9   -7.251338792831       -7.28       -4.10    1.0
 10   -7.251338797786       -8.30       -4.63    1.0
 11   -7.251338798408       -9.21       -4.73    3.0
 12   -7.251338798461      -10.28       -4.78    1.0
 13   -7.251338798695       -9.63       -5.63    2.0
 14   -7.251338798700      -11.26       -5.63    3.0
 15   -7.251338798702      -11.70       -5.80    1.0
 16   -7.251338798704      -11.67       -6.65    1.0
 17   -7.251338798704      -12.59       -6.57 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03529762025631986
[ Info: Arnoldi iteration step 2: normres = 0.4890794729952197
[ Info: Arnoldi iteration step 3: normres = 0.4518747559661879
[ Info: Arnoldi iteration step 4: normres = 0.9415737556927471
[ Info: Arnoldi iteration step 5: normres = 0.29480021902820414
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.39e-01, 5.56e-02, 1.80e-01, 1.77e-01, 2.43e-02)
[ Info: Arnoldi iteration step 6: normres = 0.537803986930005
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.28e-02, 1.64e-01, 4.86e-01, 1.06e-01, 7.65e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09855267449886945
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.46e-03, 3.86e-02, 1.75e-02, 3.13e-02, 7.19e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11094503888043547
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.68e-0